## Exception Handling

>* 예외는 오직 한 가지 일을 하는 함수의 한 부분이어야 합니다

### Bad exception handling case

In [1]:
class DataTransport:
    """An example of an object badly handling exceptions of different levels."""

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self.retry_n_times} times"
        )

    def send(self, data):
        return self.connection.send(data)
    

* 위의 deliver_event()가 다루고 있는 두 개의 예외처리 (ConnectionError, ValueError)는 서로 관련이 없으며 에러 발생시 책임의 소재가 불분명하다
* 따라서 각 예외처리는 세분화된 메소드로 나누어 그 안에서 다뤄지는 것이 옳다

### Better exception handling case

In [2]:
def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>.

    If it can connect, returns the connection object.
    If it's not possible after the retries, raises ConnectionError

    :param connector:           An object with a `.connect()` method.
    :param retry_n_times int:   The number of times to try to call
                                ``connector.connect()``.
    :param retry_threshold int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_threshold
            )
            time.sleep(retry_threshold)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc


class DataTransport:
    """An example of an object that separates the exception handling by
    abstraction levels.
    """

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)

    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

* 위의 코드에서 ConnectionError는 연결 구현 부분인 connect_with_retry()에서만 다뤄지며, ValueError는 보낼 event를 decode하는 send()에서만 다뤄진다
* 에러 발생시 책임의 소재가 분명하다

## Inheritance

>* 상속의 가장 주된 위험은 결합력(coupling)의 증가이다

>* 단지 부모 클래스에 있는 메서드를 공짜로 얻을 수 있기 때문에 상속을 하는 것은 좋지 않은 생각이다

### Bad inheritance case

In [3]:
import collections
from datetime import datetime


class TransactionalPolicy(collections.UserDict):
    def change_in_policy(self, customer_id, **new_policy_data):
        self[customer_id].update(**new_policy_data)
        
class TestPolicy():
    def test_get_policy(self):
        policy = TransactionalPolicy(
            {
                "client001": {
                    "fee": 1000.0,
                    "expiration_date": datetime(2020, 1, 3),
                }
            }
        )
        self.assertDictEqual(
            policy["client001"],
            {"fee": 1000.0, "expiration_date": datetime(2020, 1, 3)},
        )

        policy.change_in_policy(
            "client001", expiration_date=datetime(2020, 1, 4)
        )
        self.assertDictEqual(
            policy["client001"],
            {"fee": 1000.0, "expiration_date": datetime(2020, 1, 4)},
        )


In [5]:
policy = TransactionalPolicy({'client001': {'fee': 1000.0, 'expiration_date': datetime(2020,1,3)}})
dir(policy)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'change_in_policy',
 'clear',
 'copy',
 'data',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

* 특정 고객의 레코드에 상수 시간에 접근할 수 있는 사전 타입의 객체를 생각할 수 있다
* 하지만 이는 아래와 같은 2가지 문제점이 있다
    * 첫째, 계층구조가 명확하지 않다 --> TransactionalPolicy 이름만 보고 사전 타입인 것을 알 수 없다
    * 둘째, 불필요한 메서드들이 추가된다 --> 결합력이 증가한다
* 이에 대한 해결책은 컴포지션을 사용하는 것이다 --> 사전이 되는 것이 아니라, 사전을 활용하는 것이다!

### Better Inheritance Case

In [7]:
from datetime import datetime

class TransactionalPolicy:
    def __init__(self, policy_data, **extra_data):
        self._data = {**policy_data, **extra_data}
    def change_in_policy(self, customer_id, **new_policy_data):
        self._data[customer_id].update(**new_policy_data)
    def __getitem__(self, customer_id):
        return self._data[customer_id]
    def __len__(self):
        return len(self._data)
    
class TestPolicy():
    def test_get_policy(self):
        policy = TransactionalPolicy(
            {
                "client001": {
                    "fee": 1000.0,
                    "expiration_date": datetime(2020, 1, 3),
                }
            }
        )
        self.assertDictEqual(
            policy["client001"],
            {"fee": 1000.0, "expiration_date": datetime(2020, 1, 3)},
        )

        policy.change_in_policy(
            "client001", expiration_date=datetime(2020, 1, 4)
        )
        self.assertDictEqual(
            policy["client001"],
            {"fee": 1000.0, "expiration_date": datetime(2020, 1, 4)},
        )

In [9]:
policy_better = TransactionalPolicy({'client001': {'fee': 1000.0, 'expiration_date': datetime(2020,1,3)}})
dir(policy_better)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data',
 'change_in_policy']

* 위의 방법은 개념적으로도 정확할 뿐만 아니라 확장성도 뛰어나다
    * 현재 사전인 데이터 구조를 향후 변경하려 해도 인터페이스만 유지하면 사용자는 영향을 받지 않는다
    * 이는 결합력을 줄이고 파급 효과를 최소화하여 unittest를 변경하지 않아도 된다

## Acronyms to live by

### DRY (Do not Repeat Yourself)

* 코드 중복은 아래와 같은 이유로 유지보수에 치명적이다
    * 오류가 발생하기 쉽다 --> 수정시에 하나라도 빠뜨리면 버그가 발생한다
    * 비용이 비싸다 --> 여러군데에 정의한 것을 고치기 위한 많은 비용과 노력이 발생한다
    * 신뢰성이 떨어진다 

### Keep it Simple (KIS)

#### Unnecessarily Complicated Case

In [3]:
class ComplicatedNamespace:
    ACCEPTED_VALUES = {'id_', 'user', 'location'}
    @classmethod
    def init_with_data(cls, **data):
        instance = cls()
        for key, value in data.items():
            setattr(instance, key, value)
        return instance

* 위의 예제는 객체를 초기화하기 위해 추가 클래스 메소드(init_with_data)를 정의한 부분이다
    * 이는 불필요하며, 사용자에게 일반적이지 않은 이름의 메소드를 강요하고, 반복을 통해 setattr를 호출하는 것은 더욱 더 에러이다

#### Simple Case

In [4]:
class SimpleNamespace:
    ACCEPTED_VALUES = {'id_', 'user', 'location'}
    
    def __init__(self, **data):
        accepted_data = {k: v for k, v in data.items() if k in self.ACCEPTED_VALUES}
        self.__dict__.update(accepted_data)

### EAFP (Easier to Ask Forgiveness than Permission) & LBYL (Look Before You Leap)

* EAFP는 일단 코드를 실행하고 실제 동작하지 않을 경우에 대응한다는 뜻이다 (try, catch, except)
* LBYL은 그 반대로, 무엇을 사용하려고 하는지 먼저 확인하라는 뜻이다 (if os.path.exists...)
* Python은 EAFP의 방식으로 만들어졌다

## Orthogonality